In [98]:
import pandas as pd
import csv as csv
import matplotlib.pyplot as plt
from config import gkey 
import gmaps
import requests
import json
import numpy as np
from ipywidgets.embed import embed_minimal_html

In [67]:
init_traffic_data = pd.read_csv("traffic_accidents.csv")
traffic_cam_data = pd.read_csv('intelligent_traffic_system_devices.csv')

In [68]:
#dropping unneeded columns
traffic_data = init_traffic_data.drop(labels=["GEO_X","GEO_Y","OBJECTID_1","INCIDENT_ID","OFFENSE_ID","LAST_OCCURENCE_DATE","REPORTED_DATE","OFFENSE_TYPE_ID","OFFENSE_CATEGORY_ID"],axis=1)

In [69]:
#renaming columns a *little* better
traffic_data.rename(columns={"FIRST_OCCURENCE_DATE":"ACCIDENT_DATE","GEO_LON":"LON","GEO_LAT":"LAT"},inplace=True)
traffic_data.head()

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,ACCIDENT_DATE,INCIDENT_ADDRESS,LON,LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
0,5401,0,2012-11-02 18:20:00,1900 BLOCK S SHERIDAN BLVD,-105.052849,39.682112,4,421,Harvey Park,0,0
1,5441,0,2012-11-05 14:16:00,E HAMPDEN AVE / S YOSEMITE ST,-104.884786,39.653067,3,323,Hampden South,0,0
2,5401,0,2012-04-02 17:00:00,1300 BLOCK N BROADWAY ST,-104.986886,39.737147,6,623,Capitol Hill,0,0
3,5441,0,2012-06-11 17:05:00,600 BLOCK E EXPOSITION AVE,-104.979833,39.703898,3,311,Washington Park West,0,0
4,5441,0,2012-03-23 11:01:00,I25 HWYSB / W COLFAX AVE,-105.014162,39.740439,1,121,Lincoln Park,0,0


In [70]:
#splitting date&time stamp to usable columns
date_split = traffic_data["ACCIDENT_DATE"].str.split(" ",expand=True,n=1)
traffic_data["DATE"] = date_split[0]
traffic_data["TIME"] = date_split[1]
date_split = traffic_data["DATE"].str.split("-",expand=True,n=2)
traffic_data["YEAR"] = date_split[0]
traffic_data["MONTH"] = date_split[1]
traffic_data["DAY"] = date_split[2]
traffic_data.drop(labels=["ACCIDENT_DATE","DATE"],axis=1,inplace=True)
traffic_data['OVERALL_OFFENSE_CODE'] = traffic_data['OFFENSE_CODE'].map(str) + traffic_data['OFFENSE_CODE_EXTENSION'].map(str)

#dropping 2019 data as it's not a full year
traffic_data = traffic_data[traffic_data['YEAR'] != "2019"]

traffic_data.head()

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,INCIDENT_ADDRESS,LON,LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND,TIME,YEAR,MONTH,DAY,OVERALL_OFFENSE_CODE
0,5401,0,1900 BLOCK S SHERIDAN BLVD,-105.052849,39.682112,4,421,Harvey Park,0,0,18:20:00,2012,11,02,54010
1,5441,0,E HAMPDEN AVE / S YOSEMITE ST,-104.884786,39.653067,3,323,Hampden South,0,0,14:16:00,2012,11,05,54410
2,5401,0,1300 BLOCK N BROADWAY ST,-104.986886,39.737147,6,623,Capitol Hill,0,0,17:00:00,2012,04,02,54010
3,5441,0,600 BLOCK E EXPOSITION AVE,-104.979833,39.703898,3,311,Washington Park West,0,0,17:05:00,2012,06,11,54410
4,5441,0,I25 HWYSB / W COLFAX AVE,-105.014162,39.740439,1,121,Lincoln Park,0,0,11:01:00,2012,03,23,54410


In [71]:
#readding cleaned up offense descriptions, since they're way more useful than a code

descriptionlist = []
for eachcode in traffic_data['OVERALL_OFFENSE_CODE']:
    if eachcode == "54010":
        descriptionlist.append("Hit and Run")
    elif eachcode =="54200":
        descriptionlist.append("DUI")
    elif eachcode =="54410":
        descriptionlist.append("Routine Accident")
    elif eachcode =="54411":
        descriptionlist.append("Serious Bodily Injury")
    elif eachcode =="54412":
        descriptionlist.append("Fatal")
    elif eachcode =="54413":
        descriptionlist.append("Police")

traffic_data['OFFENSE_DESCRIPTION'] = descriptionlist

In [72]:
#reorder the dataframe, final usable dataframe
traffic_data = traffic_data[['OVERALL_OFFENSE_CODE','OFFENSE_DESCRIPTION','YEAR','MONTH',
                             'DAY','TIME','LAT','LON','INCIDENT_ADDRESS','DISTRICT_ID','PRECINCT_ID',
                             'NEIGHBORHOOD_ID','BICYCLE_IND','PEDESTRIAN_IND']]
traffic_data.head()

,OVERALL_OFFENSE_CODE,OFFENSE_DESCRIPTION,YEAR,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
0,54010,Hit and Run,2012,11,02,18:20:00,39.682112,-105.052849,1900 BLOCK S SHERIDAN BLVD,4,421,Harvey Park,0,0
1,54410,Routine Accident,2012,11,05,14:16:00,39.653067,-104.884786,E HAMPDEN AVE / S YOSEMITE ST,3,323,Hampden South,0,0
2,54010,Hit and Run,2012,04,02,17:00:00,39.737147,-104.986886,1300 BLOCK N BROADWAY ST,6,623,Capitol Hill,0,0
3,54410,Routine Accident,2012,06,11,17:05:00,39.703898,-104.979833,600 BLOCK E EXPOSITION AVE,3,311,Washington Park West,0,0
4,54410,Routine Accident,2012,03,23,11:01:00,39.740439,-105.014162,I25 HWYSB / W COLFAX AVE,1,121,Lincoln Park,0,0


In [73]:
#group by year
traffic_data_groupby_year = traffic_data.groupby(by="YEAR")
traffic_data_groupby_year.count()

,OVERALL_OFFENSE_CODE,OFFENSE_DESCRIPTION,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
YEAR,,,,,,,,,,,,,
2012,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560,20560
2013,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182,21182
2014,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444,22444
2015,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842,23842
2016,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254,24254
2017,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617,24617
2018,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905,24905


In [74]:
#group by offense description
traffic_data_groupby_offense = traffic_data.groupby(by=["OFFENSE_DESCRIPTION"])
traffic_data_groupby_offense.count()

,OVERALL_OFFENSE_CODE,YEAR,MONTH,DAY,TIME,LAT,LON,INCIDENT_ADDRESS,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,BICYCLE_IND,PEDESTRIAN_IND
OFFENSE_DESCRIPTION,,,,,,,,,,,,,
DUI,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030,5030
Fatal,331,331,331,331,331,331,331,331,331,331,331,331,331
Hit and Run,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112,43112
Police,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570,1570
Routine Accident,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538,109538
Serious Bodily Injury,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223,2223


In [75]:
#group by offense description THEN year
traffic_data_groupby_offenseandyear = traffic_data.groupby(by=["OFFENSE_DESCRIPTION","YEAR"])
traffic_data_groupby_offenseandyear.count()

OVERALL_OFFENSE_CODE  MONTH    DAY   TIME    LAT  \
OFFENSE_DESCRIPTION   YEAR                                                     
DUI                   2012                   707    707    707    707    707   
                      2013                   731    731    731    731    731   
                      2014                   815    815    815    815    815   
                      2015                   779    779    779    779    779   
                      2016                   692    692    692    692    692   
                      2017                   608    608    608    608    608   
                      2018                   698    698    698    698    698   
Fatal                 2012                    33     33     33     33     33   
                      2013                    43     43     43     43     43   
                      2014                    44     44     44     44     44   
                      2015                    54     54     54     54     54   
                      2016                    50     50     50     50     50   
                      2017                    48     48     48     48     48   
                      2018                    59     59     59     59     59   
Hit and Run           2012                  6129   6129   6129   6129   6129   
                      2013                  5359   5359   5359   5359   5359   
                      2014                  5577   5577   5577   5577   5577   
                      2015                  5933   5933   5933   5933   5933   
                      2016                  6444   6444   6444   6444   6444   
                      2017                  6809   6809   6809   6809   6809   
                      2018                  6861   6861   6861   6861   6861   
Police                2012                   173    173    173    173    173   
                      2013                   236    236    236    236    236   
                      2014                   186    186    186    186    186   
                      2015                   236    236    236    236    236   
                      2016                   238    238    238    238    238   
                      2017                   257    257    257    257    257   
                      2018                   244    244    244    244    244   
Routine Accident      2012                 13284  13284  13284  13284  13284   
                      2013                 14518  14518  14518  14518  14518   
                      2014                 15474  15474  15474  15474  15474   
                      2015                 16506  16506  16506  16506  16506   
                      2016                 16476  16476  16476  16476  16476   
                      2017                 16583  16583  16583  16583  16583   
                      2018                 16697  16697  16697  16697  16697   
Serious Bodily Injury 2012                   234    234    234    234    234   
                      2013                   295    295    295    295    295   
                      2014                   348    348    348    348    348   
                      2015                   334    334    334    334    334   
                      2016                   354    354    354    354    354   
                      2017                   312    312    312    312    312   
                      2018                   346    346    346    346    346   

                              LON  INCIDENT_ADDRESS  DISTRICT_ID  PRECINCT_ID  \
OFFENSE_DESCRIPTION   YEAR                                                      
DUI                   2012    707               707          707          707   
                      2013    731               731          731          731   
                      2014    815               815          815          815   
                      2015    779               779          779          779   
                     

In [76]:
#group by offense year THEN description
traffic_data_groupby_yearandoffense = traffic_data.groupby(by=["YEAR","OFFENSE_DESCRIPTION"])
traffic_data_groupby_yearandoffense.count()

OVERALL_OFFENSE_CODE  MONTH    DAY   TIME    LAT  \
YEAR OFFENSE_DESCRIPTION                                                       
2012 DUI                                     707    707    707    707    707   
     Fatal                                    33     33     33     33     33   
     Hit and Run                            6129   6129   6129   6129   6129   
     Police                                  173    173    173    173    173   
     Routine Accident                      13284  13284  13284  13284  13284   
     Serious Bodily Injury                   234    234    234    234    234   
2013 DUI                                     731    731    731    731    731   
     Fatal                                    43     43     43     43     43   
     Hit and Run                            5359   5359   5359   5359   5359   
     Police                                  236    236    236    236    236   
     Routine Accident                      14518  14518  14518  14518  14518   
     Serious Bodily Injury                   295    295    295    295    295   
2014 DUI                                     815    815    815    815    815   
     Fatal                                    44     44     44     44     44   
     Hit and Run                            5577   5577   5577   5577   5577   
     Police                                  186    186    186    186    186   
     Routine Accident                      15474  15474  15474  15474  15474   
     Serious Bodily Injury                   348    348    348    348    348   
2015 DUI                                     779    779    779    779    779   
     Fatal                                    54     54     54     54     54   
     Hit and Run                            5933   5933   5933   5933   5933   
     Police                                  236    236    236    236    236   
     Routine Accident                      16506  16506  16506  16506  16506   
     Serious Bodily Injury                   334    334    334    334    334   
2016 DUI                                     692    692    692    692    692   
     Fatal                                    50     50     50     50     50   
     Hit and Run                            6444   6444   6444   6444   6444   
     Police                                  238    238    238    238    238   
     Routine Accident                      16476  16476  16476  16476  16476   
     Serious Bodily Injury                   354    354    354    354    354   
2017 DUI                                     608    608    608    608    608   
     Fatal                                    48     48     48     48     48   
     Hit and Run                            6809   6809   6809   6809   6809   
     Police                                  257    257    257    257    257   
     Routine Accident                      16583  16583  16583  16583  16583   
     Serious Bodily Injury                   312    312    312    312    312   
2018 DUI                                     698    698    698    698    698   
     Fatal                                    59     59     59     59     59   
     Hit and Run                            6861   6861   6861   6861   6861   
     Police                                  244    244    244    244    244   
     Routine Accident                      16697  16697  16697  16697  16697   
     Serious Bodily Injury                   346    346    346    346    346   

                              LON  INCIDENT_ADDRESS  DISTRICT_ID  PRECINCT_ID  \
YEAR OFFENSE_DESCRIPTION                                                        
2012 DUI                      707               707          707          707   
     Fatal                     33                33           33           33   
     Hit and Run             6129              6129         6129         6129   
     Police                   173               173          173          173   
     Routine Accident

In [77]:
# traffic_data_morning_rush_hour = traffic_data[(traffic_data['TIME'] >= '05:00:00') & (traffic_data['TIME'] <= '09:00:00')]

# traffic_data_evening_rush_hour = traffic_data[(traffic_data['TIME'] >= '16:00:00') & (traffic_data['TIME'] <= '20:00:00')]

# traffic_data_midday = traffic_data[(traffic_data['TIME'] > '09:00:00') & (traffic_data['TIME'] < '16:00:00')]

# traffic_data_midday.head()

In [99]:
#print(gkey)
gmaps.configure(api_key=gkey)

#no way to export map or save map as an image as a command (at least I can't find anything in the gmaps documentation)
#so i can't just loop each year, i had to generate the map for each year individually.
#also can't title in gmaps??
fig_layout = {'width':'500px','height':'500px','border':'1px solid black'}
locations = traffic_data[traffic_data['YEAR'] == "2018"][['LAT','LON']]
fig = gmaps.figure(center=(39.72,-104.95), zoom_level=11,layout=fig_layout)
fig.add_layer(gmaps.heatmap_layer(locations,point_radius=5,max_intensity=30))
fig
embed_minimal_html('export.html', views=[fig])

In [79]:
# traffic_cam_data.head()

In [80]:
traffic_cam_data_date = pd.to_datetime(traffic_cam_data['RECORD_DATE'])
traffic_cam_data['DATE'] = traffic_cam_data_date
# traffic_cam_data.head()

traffic_cam_data_clean_date = traffic_cam_data.drop(columns=['RECORD_DATE'])
traffic_cam_data_clean_date.head()
# traffic_cam_data_clean_date.dtypes

,ITS_DEVICE_ID,ITS_DEVICE_TYPE,LOCATION_DESC,ITS_DEVICE_OWNER,DATE
0,626,TRAFFICCAM,Brighton and 31st,PWTES,2016-02-25
1,627,TRAFFICCAM,Brighton and 44th,PWTES,2016-02-28
2,628,TRAFFICCAM,Colfax and Wolff,PWTES,2016-03-13
3,146,Traffic camera,15th and Welton,CCD Traffic Engineering,2012-01-09
4,147,Traffic camera,15th and Larimer,CCD Traffic Engineering,2012-01-09


In [81]:
lat = []
lng = []

target_location = 'Brighton and 44th, Denver, CO'
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
'address={0}&key={1}').format(target_location, gkey)
geo_data = requests.get(target_url).json()

### The Output for the cell below has been saved into CSV file (traffic_cam_lat_and_lng.csv). That way we don't have to run every time. 

In [90]:
# lat = []
# lng = []
# for row in traffic_cam_data_clean_date['LOCATION_DESC']:
#     target_location = row + ', Denver, CO'
#     target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
#     'address={0}&key={1}').format(target_location, gkey)
#     geo_data = requests.get(target_url).json()
#     print(row)
#     if geo_data["status"] == "OK":
#         lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
#         lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
#     else:
#         lat.append("")
#         lng.append("")
# traffic_cam_data_clean_date['Lat'] = lat
# traffic_cam_data_clean_date['Lng'] = lng
# traffic_cam_data_clean_date

Brighton and 31st
Brighton and 44th
Colfax and Wolff
15th and Welton
15th and Larimer
15th and Champa
14th and Broadway
14th and Bannock
14th and Stout
14th and Logan
14th and Kalamath
14th and Court
14th and Cherokee
14th and Arapahoe
13th and Lincoln
13th and Grant
10th and Lincoln
9th and Auraria
8th and Zuni
8th and Kalamath
7th and Auraria Pkwy
5th and Walnut
5th and Walnut
5th and Walnut
3rd and Josephine
1st and University
1st and Steele
Auraria Overpass Ramp Gore Point
Colfax and I-25
I-25 and Alameda E
I-25 and Speer
I-270 and Quebec
Florida and Santa Fe
Federal and Mississippi
23rd and Federal
Downing and MLK and Champa
Chambers GVR
I-70 and Brighton Blvd
I-70 (Stapleton Dr S) and Dahlia
I-70 and Federal
I-70 and Havana
I-70 and Holly (Stapleton Dr S)
Downing and Welton
I-25 and Speer
12th St and Tremont Pl and Fox St and Colfax Ave
15th and Stout
Northfield and Dallas
4th and Josephine
Florida and Zuni
13th and Arapahoe/Champa
12th and Bannock
11th and Yosemite
14th and Yose

Downing and Louisiana
Northfield and Uinta
38th and Irving
38th and Perry
38th and Zuni
16th and Champa
18th and Franklin
Federal and Dartmouth
17th and Chestnut
Colorado and Montview
8500 E Lowry
Sheridan and Yale 
15th and Wewatta
Park Avenue West and Tremont 
Park and 18th/Ogden
17th and Steele
Colfax and Downing
I-70 and Peoria S
Ulster and Tufts
Speer and 11th
Leetsdale and Quebec
Park Avenue West and Washington/20th
Federal and Dick Connor/17th
26th and Federal
DTC and Tufts
Colfax and Kalamath
17th and Welton
20th and Lincoln
6th and Colorado
Colorado and 6th
Colorado and Yale N I/S
32nd and Sheridan
35th and Federal
University and Exposition/Bonnie Brae
Lower Colfax and Platte River W (BN Railroad Crossing)
19th and Logan
16th and Arapahoe
40th and Colorado
Colfax and Quebec 
Colfax and Grant
Chambers and Gateway Ave/GVR (48th)
Leetsdale and Alameda
Colorado / Alameda
Alameda and Federal
7th and Auraria (A side)
9th and Auraria
20th and Blake
Broadway and Kentucky
I-25 and Fran

,ITS_DEVICE_ID,ITS_DEVICE_TYPE,LOCATION_DESC,ITS_DEVICE_OWNER,DATE,Lat,Lng
0,626,TRAFFICCAM,Brighton and 31st,PWTES,2016-02-25,39.7666,-104.984
1,627,TRAFFICCAM,Brighton and 44th,PWTES,2016-02-28,39.7784,-104.968
2,628,TRAFFICCAM,Colfax and Wolff,PWTES,2016-03-13,39.7403,-105.049
3,146,Traffic camera,15th and Welton,CCD Traffic Engineering,2012-01-09,39.7436,-104.993
4,147,Traffic camera,15th and Larimer,CCD Traffic Engineering,2012-01-09,39.7483,-104.999
5,148,Traffic camera,15th and Champa,CCD Traffic Engineering,2012-01-09,39.7456,-104.995
6,149,Traffic camera,14th and Broadway,CCD Traffic Engineering,2012-01-09,39.7384,-104.987
7,150,Traffic camera,14th and Bannock,CCD Traffic Engineering,2012-01-09,39.7384,-104.99
8,151,Traffic camera,14th and Stout,CCD Traffic Engineering,2012-01-09,39.744,-104.995
9,152,Traffic camera,14th and Logan,CCD Traffic Engineering,2012-01-09,39.7384,-104.982


In [91]:
# traffic_cam_data_clean_date.to_csv('./traffic_cam_lat_and_lng.csv')
# final_traffic_cam_data = pd.read_csv('./traffic_cam_lat_and_lng.csv').drop(columns='Unnamed: 0')
# final_traffic_cam_data.head()

,ITS_DEVICE_ID,ITS_DEVICE_TYPE,LOCATION_DESC,ITS_DEVICE_OWNER,DATE,Lat,Lng
0,626,TRAFFICCAM,Brighton and 31st,PWTES,2016-02-25,39.766576,-104.983672
1,627,TRAFFICCAM,Brighton and 44th,PWTES,2016-02-28,39.778448,-104.968431
2,628,TRAFFICCAM,Colfax and Wolff,PWTES,2016-03-13,39.740340,-105.048678
3,146,Traffic camera,15th and Welton,CCD Traffic Engineering,2012-01-09,39.743602,-104.992526
4,147,Traffic camera,15th and Larimer,CCD Traffic Engineering,2012-01-09,39.748277,-104.998625


In [100]:
gmaps.configure(api_key=gkey)
final_traffic_cam_data['Lat'].replace('', np.nan,inplace=True)
final_traffic_cam_data.dropna(axis=0,how='any',inplace=True)
cam_fig_layout = {'width':'500px','height':'500px','border':'1px solid black'}
cam_locations = final_traffic_cam_data[['Lat','Lng']]
cam_fig = gmaps.figure(center=(39.72,-104.95), zoom_level=11,layout=fig_layout)
cam_fig.add_layer(gmaps.heatmap_layer(cam_locations,point_radius=5, max_intensity=1))
cam_fig
embed_minimal_html('traffic_cam_heatmap.html', views=[cam_fig])